In [1]:
from scipy.io import loadmat
# import torch
# from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from sklearn.model_selection import train_test_split
# import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import math
# import torch.optim as optim #torch.manual_seed(42)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from sklearn.metrics import confusion_matrix, roc_curve, auc, RocCurveDisplay
import pandas as pd
# import torch_geometric.transforms as T
# from torch_geometric.nn import GCNConv
# from torch_geometric.data import Data
# from google.colab import drive
from statistics import mean, stdev
from scipy.interpolate import interp1d # Import the interp1d function
# import tikzplotlib as tk
# import optuna
# import torch
# import torch.nn.functional as F
# from torch.optim import Adam
# from torch_geometric.loader import DataLoader
from sklearn.metrics import roc_auc_score
import os
import pandas as pd
from sklearn.model_selection import KFold
import time
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_curve, auc, RocCurveDisplay

In [2]:
from importlib.metadata import version
print(version('matplotlib'))
print(version('tikzplotlib'))

3.8.2
0.10.1


In [ ]:
# import matplotlib
# import numpy as np
# import matplotlib.pyplot as plt

# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'font.size' : 11,
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

# np.random.seed(19680801)

# # example data
# mu = 100  # mean of distribution
# sigma = 15  # standard deviation of distribution
# x = mu + sigma * np.random.randn(437)

# num_bins = 50

# fig, ax = plt.subplots()

# # the histogram of the data
# n, bins, patches = ax.hist(x, num_bins, density=1)

# # add a 'best fit' line
# y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
#      np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
# ax.plot(bins, y, '--')
# ax.set_xlabel('Smarts')
# ax.set_ylabel('Probability density')
# ax.set_title(r'Histogram of IQ: $\mu=100$, $\sigma=15$')

# # Tweak spacing to prevent clipping of ylabel
# fig.tight_layout()
# fig.set_size_inches(4.7747,3.5)
# plt.savefig('histogram.pgf')

# Helper functions

In [4]:
def average_fpr(fpr_dict):
    """Calculates the averaged FPR across classes.

    Args:
        fpr_dict: A dictionary where keys are class indices
                  and values are FPR arrays of different lengths.

    Returns:
        mean_fpr: An array representing the averaged FPR across classes.
    """

    all_fpr = []
    for class_idx, fpr in fpr_dict.items():
      fpr = fpr.tolist()
      all_fpr.append(fpr)
    max_len = max(len(fpr) for fpr in all_fpr)
    interpolated_fpr = []
    for fpr in all_fpr:
        # Create interpolation function
        f = interp1d(np.linspace(0, 1, len(fpr)), fpr, kind='linear', fill_value="extrapolate")

        # Interpolate to the maximum length
        new_fpr = f(np.linspace(0, 1, max_len))
        interpolated_fpr.append(new_fpr)

    # Calculate the mean FPR across classes
    mean_fpr = np.mean(interpolated_fpr, axis=0)

    return mean_fpr

In [5]:
import matplotlib.pyplot as pyplt
import matplotlib as plt
import matplotlib.gridspec as gridspec
def assign_model_colors(model_names):
    colors = pyplt.cm.get_cmap("Paired", len(model_names))  # Use a colormap with unique colors
    return {model: colors(i) for i, model in enumerate(model_names)}

def plot_mean_roc_fc(data):
  matplotlib.use("pgf")
  matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.size' : 20,
    'text.usetex': True,
    'pgf.rcfonts': False,
  })
  fc_metricses = list(data.keys())  # FC metric names
  model_nameses = list(next(iter(data.values())).keys())  # Model names

  fig, axes = pyplt.subplots(2, 3, figsize=(15, 10))  # 2 rows, 3 columns
  fig.patch.set_facecolor('white')  # Set the background color of the figure
  pyplt.subplots_adjust(hspace=0.5)
  fig.delaxes(axes[1,2])
  for idx, fc_name in enumerate(fc_metricses):

    ax = axes[idx // 3, idx % 3]
    ax.set_title(fc_name)

    model_colors = assign_model_colors(model_nameses)
    for model_name in model_nameses:
      fpr  = average_fpr(data[fc_name][model_name]["FPR"])
      tpr  = average_fpr(data[fc_name][model_name]["TPR"])
      auc_data = next(iter(data[fc_name][model_name]["AUC"].values()))
      if fc_name == "Spectral Granger":
        fc_name_disp = "SpcG"
      else:
       fc_name_disp = fc_name

      if model_name == "SVC":
        model_name_disp = "SVM"
      elif model_name == "GCN":
        model_name_disp = "GNN"
      else:
        model_name_disp = model_name

      if auc_data >= 0.6:
        ax.plot(fpr, tpr, label=f'{model_name_disp}', linewidth=4, color=model_colors[model_name])

    ax.plot([0, 1], [0, 1], linestyle='--', color='gray', linewidth=2)  # Diagonal line

    ax.set_xlabel('1-Specificity')
    ax.set_ylabel('Sensitivity')
    ax.legend(frameon=False)  # Remove the box around the legend
    ax.grid(True)

  pyplt.tight_layout(rect=[0, 0, 1, 0.96])

  pyplt.savefig('mean_roc_by_fc.pgf')
  pyplt.show()


In [6]:
def assign_fc_colors(fc_names):
    colors = pyplt.cm.get_cmap("Paired", len(fc_names))  # Use a colormap with unique colors
    return {model: colors(i) for i, model in enumerate(fc_names)}

def plot_mean_roc_model(data):
  # plt.use("pgf")
  plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.size' : 20,
    'text.usetex': True,
    'pgf.rcfonts': False,
  })

  fc_metricses = list(data.keys())  # FC metric names
  model_nameses = list(next(iter(data.values())).keys())  # Model names

  fig, axes = pyplt.subplots(2, 3, figsize=(15, 10))  # 2 rows, 3 columns
  fig.patch.set_facecolor('white')  # Set the background color of the figure
  gs = gridspec.GridSpec(2, 3, height_ratios=[1.5, 1])
  pyplt.subplots_adjust(hspace=0.5)
  # fig.delaxes(axes[1,2])
  # fig.suptitle('ROC Curves for Different FC Metrics', fontsize=16)
  # print(data)
  for idx, model_name in enumerate(model_nameses):
    ax = axes[idx // 3, idx % 3]  # Determine subplot position
    ax.set_title(model_name)
    fc_nameses = [fc_name for fc_name in fc_metricses]
    fc_colors = assign_fc_colors(fc_nameses)
    for fc_name in fc_metricses:
      # print(fc_name)
      #print(data[fc_name][model_name][0])
      fpr  = average_fpr(data[fc_name][model_name]["FPR"])
      # print('fpr')
      # print(fpr)
      tpr  = average_fpr(data[fc_name][model_name]["TPR"])
      auc_data = next(iter(data[fc_name][model_name]["AUC"].values()))
      if fc_name == "Spectral Granger":
        fc_name_disp = "SpcG"
      elif fc_name == "GCN":
        fc_name_disp = "GNN"
      elif fc_name == "SVC":
        fc_name_disp = "SVM"
      else:
        fc_name_disp = fc_name
      if auc_data >= 0.6:
        ax.plot(fpr, tpr, label=f'{fc_name_disp}', linewidth=4, color=fc_colors[fc_name])
    ax.plot([0, 1], [0, 1], linestyle='--', color='gray', linewidth=2)  # Diagonal line

    ax.set_xlabel('1-Specificity')
    ax.set_ylabel('Sensitivity')
    ax.legend()
    ax.grid(True)

  pyplt.tight_layout(rect=[0, 0, 1, 0.96])

  pyplt.savefig('mean_roc_by_model.pgf')
  pyplt.show()

In [7]:
import os
os.getcwd()

'c:\\Users\\anhtn\\OneDrive - PennO365\\Documents\\GitHub\\eeg-functional-connectivity-classification\\MLforFC_classification_project\\exploratory'

In [8]:
import json
import numpy as np
import glob  # To handle multiple files

# List of JSON file paths
json_files = [ "\\output_data-2025-GCN.json","\\output_data-2025-CNN.json", "\\output_data-2025-MLP.json", "\\output_data-2025-SK.json"]  # Replace with actual filenames

# Initialize a dictionary to store the combined data
combined_data = {}

# Function to convert lists to NumPy arrays
def convert_to_numpy(obj):
    if isinstance(obj, list):
        return np.array(obj) if all(isinstance(i, (int, float, list)) for i in obj) else [convert_to_numpy(i) for i in obj]
    elif isinstance(obj, dict):
        return {key: convert_to_numpy(value) for key, value in obj.items()}
    else:
        return obj  # Return as is

# Load and merge all JSON files
for file in json_files:
    with open(os.getcwd()+file, "r") as f:
        loaded_data = json.load(f)
        loaded_data = convert_to_numpy(loaded_data)  # Convert lists to NumPy arrays

    # Merge into the combined dictionary
    for FC_name, models in loaded_data.items():
        if FC_name not in combined_data:
            combined_data[FC_name] = {}

        for model_name, metrics in models.items():
            if model_name not in combined_data[FC_name]:
                combined_data[FC_name][model_name] = {}

            for metric_name, values in metrics.items():
                if metric_name not in combined_data[FC_name][model_name]:
                    combined_data[FC_name][model_name][metric_name] = {}

                # Append data to the respective metric
                combined_data[FC_name][model_name][metric_name].update(values)

print(combined_data)

{'Spectral Granger': {'GCN': {'best_params': {'N-GCN': 3, 'GCN-hidden': 128, 'DropEdge': 0.30000000000000004, 'Dropout': 0.2, 'Learning Rate': 0.001, 'Batch Size': 32}, 'FPR': {'0': array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.     

In [9]:
# model_names_torch = ["GCN"]
plot_mean_roc_fc(combined_data)
plot_mean_roc_model(combined_data)

C:\Users\anhtn\AppData\Local\Temp\ipykernel_1836\4207035376.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = pyplt.cm.get_cmap("Paired", len(model_names))  # Use a colormap with unique colors
C:\Users\anhtn\AppData\Local\Temp\ipykernel_1836\4207035376.py:59: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  pyplt.show()
C:\Users\anhtn\AppData\Local\Temp\ipykernel_1836\931861724.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = pyplt.cm.get_cmap("Paired", len(fc_names))  # Use a colormap with unique colors
C:\Users\anhtn\AppData\Local\Temp\ipykernel_1836\931861724.py:58: UserWarning: FigureCanvasPgf